# Basic Transformations

As part of this section we will see basic transformations we can perform on top of Data Frames such as filtering, aggregations, joins etc using SQL. We will build end to end solution by taking a simple problem statement.

* Spark SQL – Overview
* Define Problem Statement
* Preparing Tables
* Projecting Data
* Filtering Data
* Joining Tables - Inner
* Joining Tables - Outer
* Perform Aggregations
* Sorting Data
* Conclusion - Final Solution
Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.



If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

val username = System.getProperty("user.name")


In [ ]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Basic Transformations").
    master("yarn").
    getOrCreate

## Spark SQL – Overview

Let us get an overview of Spark SQL.

Here are the standard operations which we typically perform as part of processing the data. In Spark we can perform these using Data Frame APIs or **Spark SQL**.


* Selection or Projection – select clause
  * It is also called as row level transformations.
  * Apply standardization rules (convert names and addresses to upper case).
  * Mask partial data (SSN and Date of births).
* Filtering data – where clause
  * Get orders based on date or product or category.
* Joins – join (supports outer join as well)
  * Join multiple data sets.
* Aggregations – group by and aggregations with support of functions such as sum, avg, min, max etc
  * Get revenue for a given order
  * Get revenue for each order
  * Get daily revenue
* Sorting – order by
  * Sort the final output by date.
  * Sort the final output by date, then by revenue in descending order.
  * Sort the final output by state or province, then by revenue in descending order.
* Analytics Functions – aggregations, ranking and windowing functions
  * Get top 5 stores by revenue for each state.
  * Get top 5 products by revenue in each category.

## Define Problem Statement

Let us define problemt statement to get an overview of basic transformations using Spark SQL.
* Get Daily Product Revenue using orders and order_items data set.
* We have following fields in **orders**.
  * order_id
  * order_date
  * order_customer_id
  * order_status
* We have following fields in **order_items**.
  * order_item_id
  * order_item_order_id
  * order_item_product_id
  * order_item_quantity
  * order_item_subtotal
  * order_item_product_price
* We have one to many relationship between orders and order_items.
* **orders.order_id** is **primary key** and **order_items.order_item_order_id** is foreign key to **orders.order_id**.
* By the end of this module we will explore all standard transformation and get daily product revenue using following fields.
  * **orders.order_date**
  * **order_items.order_item_product_id**
  * **order_items.order_item_subtotal** (aggregated using date and product_id).
* We will consider only **COMPLETE** or **CLOSED** orders.

## Preparing Tables

Let us prepare the tables to solve the problem.

* Make sure database is created.
* Create **orders** table.
* Load data from local path **/data/retail_db/orders** into newly created **orders** table.
* Preview data and get count from **orders**
* Create **order_items** table.
* Load data from local path **/data/retail_db/order_items** into newly created **orders** table.
* Preview data and get count from **order_items**

As tables and data are ready let us get into how to write queries against tables to perform basic transformation.

## Projecting Data

Let us understand different aspects of projecting data. We primarily using `SELECT` to project the data.

* We can project all columns using `*` or some columns using column names.
* We can provide aliases to a column or expression using `AS` in `SELECT` clause.
* `DISTINCT` can be used to get the distinct records from selected columns. We can also use `DISTINCT *` to get unique records using all the columns.
* As of now **Spark SQL** does not support projecting all but one or few columns. It is supported in Hive. Following will work in hive and it will project all the columns from orders except for order_id.

```
SET hive.support.quoted.identifiers=none;
SELECT `(order_id)?+.+` FROM orders;
```

In [ ]:
val username = System.getProperty("user.name")
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Basic Transformations").
    master("yarn").
    getOrCreate

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

## first we are making table and other thing  with sql then with spark sql 


%%sql
<!-- it will drop all database along with table  -->
DROP DATABASE itversity_retail CASCADE;
CREATE DATABASE IF NOT EXISTS itversity_retail;
USE itversity_retail;
CREATE TABLE orders (
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',';

<!--  create order table hive  -->

In [ ]:
import sys.process._
val username = System.getProperty("user.name")
s"hdfs dfs -ls /user/itversity/warehouse/${username}_retail.db/order_items"!

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/order_items' INTO TABLE order_items

In [ ]:
import sys.process._
val username = System.getProperty("user.name")
s"hdfs dfs -ls /user/itversity/warehouse/${username}_retail.db/order_items"!

In [ ]:
%%sql

SELECT * FROM order_items LIMIT 10

In [ ]:
%%sql

SELECT count(1) FROM order_items

### same thing with spark sql 

In [ ]:
spark.sql("DROP DATABASE itversity_retail CASCADE").show()

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS itversity_retail").show()

In [ ]:
spark.sql("USE itversity_retail").show()
spark.sql("SHOW tables").show()
spark.sql("DROP TABLE orders").show()
spark.sql("DROP TABLE orders").show()

In [ ]:
spark.sql("""
CREATE TABLE orders (
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
""")

In [ ]:
import sys.process._
val username = System.getProperty("user.name")
s"hdfs dfs -ls /user/itversity/warehouse/${username}_retail.db/orders"!

In [ ]:
spark.sql("LOAD DATA LOCAL INPATH '/data/retail_db/orders' INTO TABLE orders")

In [ ]:
import sys.process._
val username = System.getProperty("user.name")
s"hdfs dfs -ls /user/itversity/warehouse/${username}_retail.db/orders"!

In [ ]:
spark.sql("SELECT * FROM orders LIMIT 10").show()
spark.sql("SELECT count(1) FROM orders").show()
spark.sql("DROP TABLE order_items")

In [ ]:
spark.sql("""
CREATE TABLE order_items (
    order_item_id INT,
    order_item_order_id INT,
    order_item_product_id INT,
    order_item_quantity INT,
    order_item_subtotal FLOAT,
    order_item_product_price FLOAT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
""")

In [ ]:
import sys.process._
val username = System.getProperty("user.name")
s"hdfs dfs -ls /user/itversity/warehouse/${username}_retail.db/order_items"!

In [ ]:
spark.sql("LOAD DATA LOCAL INPATH '/data/retail_db/order_items' INTO TABLE order_items")


In [ ]:
import sys.process._
val username = System.getProperty("user.name")
s"hdfs dfs -ls /user/itversity/warehouse/${username}_retail.db/order_items"!

In [ ]:
spark.sql("SELECT * FROM order_items LIMIT 10").show()
spark.sql("SELECT count(1) FROM order_items").show()

## Filtering Data

Let us understand how we can filter the data in Spark SQL.
* We use `WHERE` clause to filter the data.
* All comparison operators such as `=`, `!=`, `>`, `<`, etc can be used to compare a column or expression or literal with another column or expression or literal.
* We can use operators such as LIKE with % and regexp_like for pattern matching.
* Boolan OR and AND can be performed when we want to apply multiple conditions.
  * Get all orders with order_status equals to COMPLETE or CLOSED. We can also use IN operator.
  * Get all orders from month 2014 January with order_status equals to COMPLETE or CLOSED
* We need to use `IS NULL` and `IS NOT NULL` to compare against null values.

In [ ]:
spark.sql("""
CREATE TABLE students (
    student_id INT,
    student_first_name STRING,
    student_last_name STRING,
    student_phone_number STRING,
    student_address STRING
) STORED AS avro
""")

In [ ]:
spark.sql("""
INSERT INTO students 
VALUES (1, 'Scott', 'Tiger', NULL, NULL)
""")

In [ ]:
spark.sql("""
SELECT * FROM students 
WHERE student_phone_number = NULL
""").show()

In [ ]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_subtotal
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND date_format(order_date, 'yyyy-MM') = '2014-01'
""").show()

In [ ]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_order_id,
    oi.order_item_subtotal
FROM orders o LEFT OUTER JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE oi.order_item_order_id IS NULL
""").show()

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
HAVING revenue >= 500
""").show()

* We can perform global aggregations as well as aggregations by key.
* Global Aggregations
  * Get total number of orders.
  * Get revenue for a given order id.
  * Get number of records with order_status either COMPLETED or CLOSED.
* Aggregations by key - using `GROUP BY`
  * Get number of orders by date or status.
  * Get revenue for each order_id.
  * Get daily product revenue (using order date and product id as keys).
* We can also use `HAVING` clause to apply filtering on top of aggregated data.
  * Get daily product revenue where revenue is greater than $500 (using order date and product id as keys).
* Rules while using `GROUP BY`.
  * We can have the columns which are specified as part of `GROUP BY` in `SELECT` clause.
  * On top of those, we can have derived columns using aggregate functions.
  * We cannot have any other columns that are not used as part of `GROUP BY` on derived column using non aggregate functions.
  * We will not be able to use aggregate functions or aliases used in the select clause as part of the where clause.
  * If we want to filter based on aggregated results, then we can leverage `HAVING` on top of `GROUP BY` (specifying `WHERE` is not an option)
* Typical query execution - FROM -> WHERE -> GROUP BY -> SELECT

* We typically perform sorting as final step.
* Sorting can be done either by using one field or multiple fields.
* We can sort the data either in ascending order or descending order by using column or expression.
* By default, the sorting order is ascendig and we can change it to descending by using `DESC`.

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
ORDER BY o.order_date,
    revenue DESC
""").show()

## ## Exercises - Basic SQL Queries

Here are some of the exercises for which you can write SQL queries to self evaluate.
Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.
val username = System.getProperty("user.name")
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Basic Transformations").
    master("yarn").
    getOrCreate
If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.


```
### Exercise 1 - Customer order count

Get order count per customer for the month of 2014 January.
* Tables - orders and customers
* Data should be sorted in descending order by count and ascending order by customer id.
* Output should contain customer_id, customer_first_name, customer_last_name and customer_order_count.
### Exercise 2 - Dormant Customers

Get the customer details who have not placed any order for the month of 2014 January.
* Tables - orders and customers
* Data should be sorted in ascending order by customer_id
* Output should contain all the fields from customers
### Exercise 3 - Revenue Per Customer

Get the revenue generated by each customer for the month of 2014 January
* Tables - orders, order_items and customers
* Data should be sorted in descending order by revenue and then ascending order by customer_id
* Output should contain customer_id, customer_first_name, customer_last_name, customer_revenue.
* If there are no orders placed by customer, then the corresponding revenue for a give customer should be 0.
* Consider only COMPLETE and CLOSED orders
### Exercise 4 - Revenue Per Category

Get the revenue generated for each category for the month of 2014 January
* Tables - orders, order_items, products and categories
* Data should be sorted in ascending order by category_id.
* Output should contain all the fields from category along with the revenue as category_revenue.
* Consider only COMPLETE and CLOSED orders
### Exercise 5 - Product Count Per Department

Get the products for each department.
* Tables - departments, categories, products
* Data should be sorted in ascending order by department_id
* Output should contain all the fields from department and the product count as product_count


# Basic DDL and DML

As part of this section we will primarily focus on basic DDL and DML using Spark Metastore.

* Create Spark Metastore Tables
* Overview of Data Types
* Adding Comments
* Loading Data into Tables - Local
* Loading Data into Tables - HDFS
* Loading Data - Append and Overwrite
* Creating External Tables
* Managed Tables vs. External Tables
* Overview of File Formats
* Dropping Tables and Databases
* Truncating Tables

## Create Spark Metastore Tables

Let us understand how to create tables in Spark Metastore. We will be focusing on syntax and semantics.

val username = System.getProperty("user.name")
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Managing Tables - Basic DDL and DML").
    master("yarn").
    getOrCreate
If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```
* Let us drop and recreate the table. We need to determine table type, file format based up on the files that will be copied to the table. If the file format is delimited text file then we need to understand field delimiter as well.
  * Managed Table
  * Text File Format
  * Field Delimiter ','
* We will create the table based on the structure of data in **/data/retail_db/orders**
* If you are using `spark-sql` make sure to end the statements with **semi-colon**. With `spark-shell` or `pyspark` make sure to use `spark.sql` to pass these commands.


In [ ]:
spark.sql("""
CREATE TABLE orders (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE
""")

In [ ]:
spark.sql("""
CREATE TABLE students (
    student_id INT,
    student_first_name STRING,
    student_last_name STRING,
    student_phone_number STRING,
    student_address STRING
) STORED AS TEXTFILE
""")

In [ ]:
spark.sql("""
INSERT INTO students VALUES 
    (3, 'Mickey', 'Mouse', '2345678901', 'A Street, One City, Some State, 12345'),
    (4, 'Bubble', 'Guppy', '6789012345', 'Bubbly Street, Guppy, La la land, 45678')
""")    

In [ ]:

spark.sql("SHOW tables").show()

* Syntactically Hive and Spark SQL are almost same.
* Go to this [hive page](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL) and review supported data types.
* Spark Metastore supports all standard data types.
  * Numeric - INT, BIGINT, FLOAT etc
  * Alpha Numeric or String - CHAR, VARCHAR, STRING
  * Date and Timestamp - DATE, TIMESTAMP
  * Special Data Types - ARRAY, STRUCT etc
  * Boolean - BOOLEAN
* If the file format is text file with special types, then we need to consider other clauses under DELIMITED ROW FORMAT (if we don't want to use default delimiters).

In [ ]:
spark.sql("""
CREATE TABLE students (
    student_id INT,
    student_first_name STRING,
    student_last_name STRING,
    student_phone_numbers ARRAY<STRING>,
    student_address STRUCT<street:STRING, city:STRING, state:STRING, zip:STRING>
) STORED AS TEXTFILE
ROW FORMAT
    DELIMITED FIELDS TERMINATED BY '\t'
    COLLECTION ITEMS TERMINATED BY ','
    MAP KEYS TERMINATED BY ':'
""")

In [ ]:
spark.sql("""
INSERT INTO students VALUES 
    (3, 'Mickey', 'Mouse', ARRAY('1234567890', '2345678901'), STRUCT('A Street', 'One City', 'Some State', '12345')),
    (4, 'Bubble', 'Guppy', ARRAY('5678901234', '6789012345'), STRUCT('Bubbly Street', 'Guppy', 'La la land', '45678'))
""")

## Adding Comments

Let us understand how to create table with comments in Hive using orders as example.

In [ ]:
spark.sql("""
CREATE TABLE orders (
  order_id STRING COMMENT 'Unique order id',
  order_date STRING COMMENT 'Date on which order is placed',
  order_customer_id INT COMMENT 'Customer id who placed the order',
  order_status STRING COMMENT 'Current status of the order'
) COMMENT 'Table to save order level details'
""")

## Loading Data into Tables - Local

Let us understand how to load data into Spark Metastore tables. We can load either from local file system or from HDFS.

In [ ]:
spark.sql("""
CREATE TABLE orders (
  order_id INT COMMENT 'Unique order id',
  order_date STRING COMMENT 'Date on which order is placed',
  order_customer_id INT COMMENT 'Customer id who placed the order',
  order_status STRING COMMENT 'Current status of the order'
) COMMENT 'Table to save order level details'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
""")
spark.sql("LOAD DATA LOCAL INPATH '/data/retail_db/orders' INTO TABLE orders")

## Loading Data into Tables - HDFS

Let us understand how we can load data from HDFS location into Spark Metastore table.

Note: spark sql and hive share metastore 
* We can use load command with-out **LOCAL** to get data from HDFS location into Spark Metastore Table.
* User running load command from HDFS location need to have write permissions on the source location as data will be moved (deleted on source and copied to Spark Metastore table)
* Make sure user have write permissions on the source location.
* First we need to copy the data into HDFS location where user have write permissions.

* If you look at **/user/training/retail_db** orders directory would have been deleted.
* Move is much faster compared to copying the files by moving blocks around, hence Hive load command from HDFS location will always try to move files.

In [ ]:
s"hadoop fs -put -f /data/retail_db/orders /user/${username}/retail_db" !

In [ ]:
spark.sql("""
LOAD DATA INPATH '/user/itversity/retail_db/orders' 
  INTO TABLE orders""")

## Loading Data - Append and Overwrite
Let us understand different approaches to load the data into Spark Metastore table.
* `INTO TABLE` will append in the existing table
* If we want to overwrite we have to specify `OVERWRITE INTO TABLE`

In [ ]:
spark.sql("""
LOAD DATA LOCAL INPATH '/data/retail_db/orders' 
  INTO TABLE orders
""")

In [ ]:
spark.sql("""
LOAD DATA LOCAL INPATH '/data/retail_db/orders' 
  OVERWRITE INTO TABLE orders
""")

## Creating External Tables

Let us understand how to create external table in Spark Metastore using orders as example. Also we will see how to load data into external table.

* We just need to add **EXTERNAL** keyword in the **CREATE** clause and **LOCATION** after **STORED AS** clause or just **LOCATION** as part of **CREATE TABLE** statement.
* We can use same LOAD commands to get data from either local file system or HDFS which we have used for Managed table.
* Once table is created we can run `DESCRIBE FORMATTED orders` to check the metadata of the table and confirm whether it is managed table or external table.
* We need to specify the location while creating external tables.

Here is the script to create external table in Spark Metastore.

In [ ]:
%%sql

CREATE EXTERNAL TABLE orders (
  order_id INT COMMENT 'Unique order id',
  order_date STRING COMMENT 'Date on which order is placed',
  order_customer_id INT COMMENT 'Customer id who placed the order',
  order_status STRING COMMENT 'Current status of the order'
) COMMENT 'Table to save order level details'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION '/user/itversity/external/retail_db/orders'

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/orders' 
  INTO TABLE orders

## Managed Tables vs. External Tables

Let us compare and contrast between Managed Tables and External Tables.

* When we say EXTERNAL and specify LOCATION or LOCATION alone as part of CREATE TABLE, it makes the table EXTERNAL.
* Rest of the syntax is same as Managed Table.
* However, when we drop **Managed Table**, it will delete metadata from metastore as well as data from HDFS.
* When we drop **External Table**, only metadata will be dropped, not the data.
* Typically we use **External Table** when same dataset is processed by multiple frameworks such as Hive, Pig, Spark etc.
* We cannot run **TRUNCATE TABLE** command against External Tables.

## Overview of File Formats
Let us go through the details about different file formats supported by STORED AS Clause.

* Go to this [page](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL) and review supported file formats.
* Supported File Formats
  * TEXTFILE
  * ORC
  * PARQUET
  * AVRO
  * SEQUENCEFILE - is not important
  * JSONFILE - only available in recent vesions of Hive.
  * and more
* We can even specify custom file formats (out of scope)

In [ ]:
%%sql

CREATE TABLE students (
    student_id INT,
    student_first_name STRING,
    student_last_name STRING,
    student_phone_numbers ARRAY<STRING>,
    student_address STRUCT<street:STRING, city:STRING, state:STRING, zip:STRING>
) STORED AS parquet

## Dropping Tables and Databases

Let us understand how to DROP Spark Metastore Tables as well as Databases.
* We can use **DROP TABLE** command to drop the table.. Let us drop orders table

* **DROP TABLE** on managed table will delete both metadata in metastore as well as data in HDFS, while **DROP TABLE** on external table will only delete metadata in metastore.
* We can drop database by using **DROP DATABASE** Command. However we need to drop all the tables in the database first.
* Here is the example to drop the database itversity_retail - `DROP DATABASE itversity_retail`
* We can also drop all the tables and databases by adding **CASCADE**.

## Truncating Tables

Let us understand how to truncate tables.
* **TRUNCATE** works only for managed tables. Only data will be deleted, structure will be retained.
* Launch Spark SQL

## Managed Tables - Exercise
Q:
Let us use NYSE data and see how we can create tables in Spark Metastore.
* Duration: **30 Minutes**
* Data Location (Local): /data/nyse_all/nyse_data
* Create a database with the name - YOUR_OS_USER_NAME_nyse
* Table Name: nyse_eod
* File Format: TEXTFILE (default)
* Review the files by running Linux commands before using data sets. Data is compressed and we can load the files as is.
* Copy one of the zip file to your home directory and preview the data. There should be 7 fields. You need to determine the delimiter.
* Field Names: stockticker, tradedate, openprice, highprice, lowprice, closeprice, volume. For example, you need to use `BIGINT` for volume not `INT`.
* Determine correct data types based on the values
* Create Managed table with default Delimiter.
> As delimiters in data and table are not same, you need to figure out how to get data into the target table.
* Make sure the data is copied into the table as per the structure defined and validate.

# DML and Partitioning

As part of this section we will continue understanding further concepts related to DML and also get into the details related to partitioning tables. With respect to DML, earlier we have seen how to use LOAD command, now we will see how to use INSERT command primarily to get query results copied into a table.

* Introduction to Partitioning
* Creating Tables using Parquet
* LOAD vs. INSERT
* Inserting Data using Stage Table
* Creating Partitioned Tables
* Adding Partitions to Tables
* Loading data into Partitions
* Inserting Data into Partitions
* Using Dynamic Partition Mode
* Exercise - Partitioned Tables

val username = System.getProperty("user.name")
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Managing Tables - DML and Partitioning").
    master("yarn").
    getOrCreate

**Unlike Hive, Spark SQL does not support Bucketing which is similar to Hash Partitioning. However, Delta Lake does. Delta Lake is 3rd party library which facilitate us additional capabilities such as ACID transactions on top of Spark Metastore tables**

Let us make sure that we have orders table with data as we will be using it to populate partitioned tables very soon.

## Introduction to Partitioning

Let us get an overview of partitioning of Spark Metastore tables.
* It is similar to list partitioning where each partition is equal to a particular value for a given column.
* Spark Metastore does not support range partitioning and bucketing. Bucketing is supported in Hive which is similar to Hash Partitioning.
* Once the table is created, we can add static partitions and then load or insert data into it.
* Spark Metastore also support creation of partitions dynamically, where partitions will be created based up on the partition column value.
* A Partitioned table can be managed or external.

## Creating Tables using Parquet

Let us create order_items table using Parquet file format. By default, the files of table using Parquet file format are compressed using Snappy algorithm.
* A table with parquet file format can be external.
* In our case we will create managed table with file format as parquet in STORED AS clause.
* We will explore INSERT to insert query results into this table of type parquet.

In [ ]:
%%sql

CREATE TABLE order_items (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) STORED AS parquet

## LOAD vs. INSERT

Let us compare and contrast LOAD and INSERT commands. These are the main approaches using which we get data into Spark Metastore tables.
* LOAD will copy the files by dividing them into blocks.
* LOAD is the fastest way of getting data into Spark Metastore tables. However, there will be minimal validations at File level. 
* There will be no transformations or validations at data level.
* If it require any transformation while getting data into Spark Metastore table, then we need to use INSERT command.
* Here are some of the usage scenarios of insert:
  * Changing delimiters in case of text file format
  * Changing file format
  * Loading data into partitioned or bucketed tables (if bucketing is supported).
  * Apply any other transformations at data level (widely used)

example of load

In [ ]:
%%sql

CREATE TABLE order_items (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) STORED AS parquet

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/order_items'
    INTO TABLE order_items

In [ ]:
import sys.process._

s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/order_items" !

## Inserting Data using Stage Table

Let us understand how to insert data into order_items with Parquet file format. 
As data is in text file format and our table is created with Parquet file format, we will not be able to use LOAD command to load the data.
Note : when format is issue we create stagging table the insert data from stage table to our original table 
* `INSERT INTO` will append data into the target table by adding new files.
* `INSERT OVERWRITE` will overwrite the data in target table by deleting the files related to old data from the directory pointed by the Spark Metastore table.

In [ ]:
%%sql

CREATE TABLE order_items_stage (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/order_items' INTO TABLE order_items_stage

In [ ]:
%%sql

INSERT INTO TABLE order_items
SELECT * FROM order_items_stage

In [ ]:
%%sql

INSERT OVERWRITE TABLE order_items
SELECT * FROM order_items_stage

## Creating Partitioned Tables

Let us understand how to create partitioned table and get data into that table.
* Earlier we have already created orders table. We will use that as reference and create partitioned table.
* We can use `PARTITIONED BY` clause to define the **column along with data type**. In our case we will use **order_month as partition column**.
* We will not be able to directly load the data into the partitioned table using our original orders data (as data is not in sync with structure).

Here is the example of creating partitioned tables in Spark Metastore.

In [ ]:
%%sql

DROP TABLE IF EXISTS orders_part;
CREATE TABLE orders_part (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) PARTITIONED BY (order_month INT)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',';
DESCRIBE orders_part;

## Adding Partitions to Tables

Let us understand how we can add static partitions to Partitioned tables in Spark Metastore.
* We can add partitions using `ALTER TABLE` command with `ADD PARTITION`.
* For each and every partition created, a subdirectory will be created using partition column name and corresponding value under the table directory.
* Let us understand how to add partitions to **orders_part** table under **itversity_retail** database.

In [ ]:
%%sql

DROP TABLE IF EXISTS orders_part;
CREATE TABLE orders_part (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) PARTITIONED BY (order_month STRING)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',';




In [ ]:
import sys.process._

s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/orders_part" !

In [ ]:
%%sql

ALTER TABLE orders_part ADD
    PARTITION (order_month=201308)
    PARTITION (order_month=201309)
    PARTITION (order_month=201310)

see patitions are now there

In [ ]:
import sys.process._

s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/orders_part" !

## Loading into Partitions

Let us understand how to use load command to load data into partitioned tables.

* We need to make sure that file format of the file which is being loaded into table is same as the file format used while creating the table.
* We also need to make sure that delimiters are consistent between files and table for text file format.
* Also data should match the criteria for the partition into which data is loaded.
* Our `/data/retail_db/orders` have data for the whole year and hence we should not load the data directly into partition.
* We need to split into files matching partition criteria and then load into the table.

To use load command to load the files into partitions we need to pre-partition the data based on partition logic. 

Here is the example of using simple shell commands to partition the data. Use command prompt to run these commands


```shell
rm -rf ~/orders
mkdir -p ~/orders

grep 2013-07 /data/retail_db/orders/part-00000 > ~/orders/orders_201307
grep 2013-08 /data/retail_db/orders/part-00000 > ~/orders/orders_201308
grep 2013-09 /data/retail_db/orders/part-00000 > ~/orders/orders_201309
grep 2013-10 /data/retail_db/orders/part-00000 > ~/orders/orders_201310
```

Let us see how we can load data into corresponding partitions. Data has to be pre-partitioned based on the partitioned column.

Note: adding  data on partions  basis via sql,spark sql

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/home/itversity/orders/orders_201307'
  INTO TABLE orders_part PARTITION (order_month=201307)

In [ ]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part" !

## Inserting Data into Partitions

Let us understand how to use insert to get data into static partitions in Spark Metastore from existing table called as orders.
* Let us recap what is covered so far related to partitioned tables.
  * We have created a table called as orders_part with order_month of type INT as partitioned column.
  * We have added 4 static partitions for 201307, 201308, 201309 and 201310 using ALTER TABLE command.
  * Once the table is created and partitions are added we have pre-processed the data to get data into the partitions using LOAD command.
* It is not practical to use LOAD command always. We typically use `INSERT` via stage table to copy data into partitioned table.
* We can pre-create partitions in partitioned tables and insert data into partitions using appropriate `INSERT `command. One need to ensure that required filter condition is applied to get the data relevant to the partition that is being populated.
* We can also create partitions dynamically which we will see as part of the next topic.

In [ ]:
%%sql

ALTER TABLE orders_part ADD PARTITION (order_month=201311)

In [ ]:
%%sql

INSERT INTO TABLE orders_part PARTITION (order_month=201311)
  SELECT * FROM orders WHERE order_date LIKE '2013-11%'

In [ ]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part" !

## Using Dynamic Partition Mode

Note: its important 
Let us understand how we can insert data into partitioned table using dynamic partition mode.
* Using dynamic partition mode we need not pre create the partitions. Partitions will be automatically created when we issue INSERT command in dynamic partition mode.
* To insert data using dynamic partition mode, we need to set the property `hive.exec.dynamic.partition` to **true**
* Also we need to set `hive.exec.dynamic.partition.mode` to **nonstrict**

Here is the example of inserting data into partitions using dynamic partition mode.

In [ ]:
%%sql

SET hive.exec.dynamic.partition.mode;

SET hive.exec.dynamic.partition.mode=nonstrict;

In [ ]:
%%sql

INSERT INTO TABLE orders_part PARTITION (order_month)
SELECT o.*, date_format(order_date, 'yyyyMM') order_month
FROM orders o
WHERE order_date >= '2013-12-01 00:00:00.0'

In [ ]:
import sys.process._

s"hdfs dfs -ls -R /user/${username}/warehouse/${username}_retail.db/orders_part" !

## Exercise - Partitioned Tables

Let us take care of this exercise related to partitioning to self evaluate our comfort level in working with partitioned tables.
* Duration: **30 Minutes**
* Use data from **/data/nyse_all/nyse_data**
* Use database **YOUR_OS_USER_NAME_nyse**
* Create partitioned table **nyse_eod_part**
* Field Names: stockticker, tradedate, openprice, highprice, lowprice, closeprice, volume
* Determine correct data types based on the values
* Create Managed table with "," as delimiter.
* Partition Field should be **tradeyear** and of type **INT** (one partition for corresponding year)
* Insert data into partitioned table using dynamic partition mode.
* Here are the steps to come up with the solution.
  * Review the files under **/data/nyse_all/nyse_data** - determine data types (For example: tradedate should be INT and volume should be BIGINT)
  * Create database **YOUR_OS_USER_NAME_nyse** (if it does not exists)
  * Create non partitioned stage table
  * Load data into non partitioned stage table
  * Validate the count and also see that data is as expected by running simple select query.
  * Create partitioned table
  * Set required properties to use dynamic partition
  * Insert data into partitioned table - here is how you can compute year from tradedate of type int `year(to_date(cast(tradedate AS STRING), 'yyyyMMdd')) AS tradeyear`
  * Run below validate commands to validate

  ### Validation
Here are the instructions to validate the results.
* Run `hdfs dfs -ls /user/YOUR_OS_USER_NAME/warehouse/YOUR_OS_USER_NAME_nyse.db/nyse_eod_part`
* Run `SHOW PARTITIONS YOUR_OS_USER_NAME_nyse.nyse_eod_part`. You should see partitions for all the years using which you have loaded the data.
* Run `SELECT count(1) FROM YOUR_OS_USER_NAME_nyse.nyse_eod_part`. The count should match the number of records in our dataset.
* You can compare with the output generated by this simple Python code which is validated in our labs.

```
import pandas as pd
import glob

path = r'/data/nyse_all/nyse_data' # use your path
all_files = glob.glob(path + "/*.txt.gz")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=None)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.shape
```

# second Basics


# Predefined Functions

Let us go through the functions that can be used while processing the data. These are typically applied on columns to get derived values from existing column values.

* Overview of Functions
* Validating Functions
* String Manipulation Functions
* Date Manipulation Functions
* Overview of Numeric Functions
* Data Type Conversion
* Handling NULL Values
* Using CASE and WHEN
* Query Example - Word Count

## Overview of Functions
Let us get overview of pre-defined functions in Spark SQL.
* We can get list of functions by running `SHOW functions`
* We can use DESCRIBE command to get the syntax and symantecs of a function - `DESCRIBE FUNCTION substr`
* Following are the categories of functions that are more commonly used.
  * String Manipulation
  * Date Manipulation
  * Numeric Functions
  * Type Conversion Functions
  * CASE and WHEN
  * and more

In [ ]:
spark.sql("SHOW functions").show(300, false)
spark.catalog.listFunctions.show(300, false)


In [ ]:
%%sql

DESCRIBE FUNCTION substr

## Validating Functions
Let us see how we can validate Spark SQL functions.

* Spark SQL follows MySQL style. To validate functions we can just use SELECT clause - e. g.: `SELECT current_date;`
* Another example - `SELECT substr('Hello World', 1, 5);`
* If you want to use Oracle style, you can create table by name dual and insert one record.
* You can also create temporary view on top of dataframe and start writing SQL Queries. We will see an example with Scala based approach. Here are the code snippets using both Scala as well as Pyspark.

**Using Scala**
```
val orders = spark.read.
    schema("order_id INT, order_date STRING, order_customer_id INT, order_status STRING").
    csv("/public/retail_db/orders")
orders.createOrReplaceTempView("orders_temp")
```

**Using Python**
```
orders = spark.read. \
    schema("order_id INT, order_date STRING, order_customer_id INT, order_status STRING"). \
    csv("/public/retail_db/orders")
orders.createOrReplaceTempView("orders_temp")
```

In [ ]:
val orders = spark.read.
    schema("order_id INT, order_date STRING, order_customer_id INT, order_status STRING").
    csv("/public/retail_db/orders")

In [ ]:
orders.createOrReplaceTempView("orders_temp")

In [ ]:
%%sql

SELECT o.*, lower(order_status) AS order_status_lower FROM orders_temp AS o LIMIT 10

## String Manipulation Functions

We use string manipulation functions quite extensively. Here are some of the important functions which we typically use.

* Case Conversion - `lower`, `upper`, `initcap`
* Getting size of the column value - `length`
* Extracting Data - `substr` and `split`
* Trimming and Padding functions - `trim`, `rtrim`, `ltrim`, `rpad` and `lpad`
* Reversing strings - `reverse`
* Concatenating multiple strings `concat` and `concat_ws`

### Case Conversion and Length
Let us understand how to perform case conversion functions of a string and also length of a string.

* Case Conversion Functions - `lower`, `upper`, `initcap`

### Extracting Data - substr and split
Let us understand how to extract data from strings using `substr`/`substring` and `split`.

* We can get syntax and symantecs of the functions using `DESCRIBE FUNCTION`
* We can extract first four characters from string using substr or substring.

Let us understand how to extract the information from the string where there is a delimiter.
* `split` converts delimited string into array.

* We can use explode to convert an array into records.

### Trimming and Padding Functions

Let us understand how to trim or remove leading and/or trailing spaces in a string.

* `ltrim` is used to remove the spaces on the left side of the string.
* `rtrim` is used to remove the spaces on the right side of the string.
* `trim` is used to remove the spaces on both sides of the string.

Let us understand how to use padding to pad characters to a string.

* Let us assume that there are 3 fields - year, month and date which are of type integer.
* If we have to concatenate all the 3 fields and create a date, we might have to pad month and date with 0.
* `lpad` is used more often than `rpad` especially when we try to build the date from separate columns.


### Reverse and Concatenating multiple strings

Let us understand how to reverse a string as well as concatenate multiple strings.
* We can use `reverse` to reverse a string.
* We can concatenate multiple strings using `concat` and `concat_ws`.
* `concat_ws` is typically used if we want to have the same string between all the strings that are being concatenated.

## Date Manipulation Functions

Let us go through some of the important date manipulation functions.

* Getting Current Date and Timestamp
* Date Arithmetic such as `date_add`
* Getting beginning date or time using `trunc` or `date_trunc`
* Extracting information using `date_format` as well as calendar functions.
* Dealing with unix timestamp using `from_unixtime`, `to_unix_timestamp`

### Getting Current Date and Timestamp

Let us understand how to get the details about current or today's date as well as current timestamp.

* `current_date` is the function or operator which will return today's date.
* `current_timestamp` is the function or operator which will return current time up to milliseconds.
* These are not like other functions and do not use **()** at the end.
* These are not listed as part of `SHOW functions` and we can get help using `DESCRIBE`.
* There is a format associated with date and timestamp.
  * Date - `yyyy-MM-dd`
  * Timestamp - `yyyy-MM-dd HH:mm:ss.SSS`
* Keep in mind that a date or timestamp in Spark SQL are nothing but special strings containing values using above specified formats. We can apply all string manipulation functions on date or timestamp.

### Date Arithmetic
Let us understand how to perform arithmetic on dates or timestamps.

* `date_add` can be used to add or subtract days.
* `date_sub` can be used to subtract or add days.
* `datediff` can be used to get difference between 2 dates
* `add_months` can be used add months to a date


### Beginning Date or Time - trunc and date_trunc
Let us understand how to use `trunc` and `date_trunc` on dates or timestamps and get beginning date of the period.

* We can use **MM** to get beginning date of the month.
* **YY** can be used to get begining date of the year.
* We can apply trunc either on date or timestamp, however we cannot apply it other than month or year (such an hour or day).

* While `trunc` can be used to  beginning time of a given month or year, we can get the beginning time up to Second using `date_trunc`.get

### Extracting information using date_format

Let us understand how to use `date_format` to extract information from date or timestamp.

Here is how we can get date related information such as year, month, day etc from date or timestamp.


Note: date_format(<column>,'yyyyMM') === to convert column to required datae format

In [ ]:
%%sql

SELECT current_timestamp AS current_timestamp, 
    date_format(current_timestamp, 'dd') AS day_of_month

* Here is how we can get the information from date or timestamp in the format we require.

In [ ]:
%%sql

SELECT date_format(current_timestamp, 'yyyyMM') AS current_month

### Extracting information - Calendar functions

We can get year, month, day etc from date or timestamp using functions.
 There are functions such as `day`, `dayofmonth`, `month`, `weekofyear`, `year` etc available for us.

%%sql

SELECT weekofyear(current_date) AS weekofyear

### Dealing with Unix Timestamp

Let us go through the functions that can be used to deal with Unix Timestamp.

* `from_unixtime` can be used to convert Unix epoch to regular timestamp.
* `unix_timestamp` or `to_unix_timestamp` can be used to convert timestamp to Unix epoch.
* We can get Unix epoch or Unix timestamp by running `date '+%s'` in Unix/Linux terminal
* We can DESCRIBE on the above functions to get details about them.

Let us sww how we can use functions such as `from_unixtime`, `unix_timestamp` or `to_unix_timestamp` to convert between timestamp and Unix timestamp or epoch.

* We can unix epoch in Unix/Linux terminal using `date '+%s'`

In [ ]:
%%sql

SELECT from_unixtime(1556662731, 'yyyy-MM-dd') AS date

In [ ]:
%%sql

SELECT to_unix_timestamp('20190430 18:18:51', 'yyyyMMdd HH:mm:ss') AS timestamp

## Overview of Numeric Functions

Here are some of the numeric functions we might use quite often.

* `abs` - always return positive number
* `sum`, `avg`
* `round` - rounds off to specified precision
* `ceil`, `floor` - always return integer.
* `greatest`
* `min`, `max`
* `rand`
* `pow`, `sqrt`
* `cumedist`, `stddev`, `variance`

Some of the functions highlighted are aggregate functions, eg: `sum`, `avg`, `min`, `max` etc.

## Data Type Conversion

Let us understand how we can type cast to change the data type of extracted value to its original type.

In [ ]:
%%sql

CREATE EXTERNAL TABLE IF NOT EXISTS orders_single_column (
    s STRING
) LOCATION '/user/itversity/warehouse/itversity_retail.db/orders'

In [ ]:
%%sql

SELECT split(s, ',')[0] AS order_id,
    split(s, ',')[1] AS order_date,
    split(s, ',')[2] AS order_customer_id,
    split(s, ',')[3] AS order_status
FROM orders_single_column LIMIT 10

## Handling NULL Values

Let us understand how to handle nulls using specific functions in Spark SQL.

* By default if we try to add or concatenate null to another column or expression or literal, it will return null.
* If we want to replace null with some default value, we can use `nvl`. For not null values, nvl returns the original expression value.
  * Replace commission_pct with 0 if it is null.
  * We can also use `coalesce` in the place of `nvl`.
  * nvl(<col>,replacement)
  * nvl2():If expr1 is not null, then NVL2 returns expr2.
* `coalesce` returns first not null value if we pass multiple arguments to it.
* `nvl2` can be used to perform one action when the value is not null and some other action when the value is null.
  * We want to increase commission_pct by 1 if it is not null and set commission_pct to 2 if it is null.
* We can also use `CASE WHEN ELSE END` for any conditional logic.


## Using CASE and WHEN
At times we might have to select values from multiple columns conditionally.

* We can use `CASE` and `WHEN` for that.
* Let us implement this conditional logic to come up with derived order_status.
  * If order_status is COMPLETE or CLOSED, set COMPLETED
  * If order_status have PENDING in it, then we will say PENDING
  * If order_status have PROCESSING or PAYMENT_REVIEW in it, then we will say PENDING
  * We will set all others as OTHER
* We can also have `ELSE` as part of `CASE` and `WHEN`.


In [ ]:
%%sql

SELECT DISTINCT order_status,
    CASE 
        WHEN order_status IN ('COMPLETE', 'CLOSED') THEN 'COMPLETED'
        WHEN order_status LIKE '%PENDING%' OR order_status IN ('PROCESSING', 'PAYMENT_REVIEW')
            THEN 'PENDING'
        ELSE 'OTHER'
    END AS updated_order_status
FROM orders
ORDER BY updated_order_status

## Query Example - Word Count

Let us see how we can perform word count using Spark SQL. Using word count as an example we will understand how we can come up with the solution using pre-defined functions available.

* Create table by name lines.
* Insert data into the table.
* Split lines into array of words.
* Explode array of words from each line into individual records.
* Use group by and get the count. We cannot use `GROUP BY` directly on exploded records and hence we need to use nested sub query.

In [ ]:
%%sql

SELECT count(1) FROM (SELECT explode(split(s, ' ')) AS words FROM lines)

# Windowing Functions

As part of this section we will primarily talk about Windowing Functions. These are also known as Analytic Functions in Databases like Oracle.

* Prepare HR Database
* Overview of Windowing Functions
* Aggregations using Windowing Functions
* Getting LEAD and LAG values
* Getting first and last values
* Ranking using Windowing Functions
* Understanding order of execution of SQL
* Overview of Nested Sub Queries
* Filtering - Window Function Results

## Prepare HR Database

Let us prepare HR database with **EMPLOYEES** Table. We will be using this for some of the examples as well as exercises related to Window Functions.

* Create Database **itversity_hr** (replace itversity with your OS User Name)
* Create table **employees** in **itversity_hr** database.
* Load data into the table.

First let us start with creating the database.

In [ ]:
%%sql

CREATE TABLE employees (
  employee_id     int,
  first_name      varchar(20),
  last_name       varchar(25),
  email           varchar(25),
  phone_number    varchar(20),
  hire_date       date,
  job_id          varchar(10),
  salary          decimal(8,2),
  commission_pct  decimal(2,2),
  manager_id      int,
  department_id   int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/hr_db/employees' 
INTO TABLE employees

## Overview of Windowing Functions

Let us get an overview of Analytics or Windowing Functions in Spark SQL.

* Aggregate Functions (`sum`, `min`, `max`, `avg`)
* Window Functions (`lead`, `lag`, `first_value`, `last_value`)
* Rank Functions (`rank`, `dense_rank`, `row_number` etc)
* For all the functions we use `OVER` clause.
* For aggregate functions we typically use `PARTITION BY`
* For global ranking and windowing functions we can use `ORDER BY sorting_column` and for ranking and windowing with in a partition or group we can use `PARTITION BY partition_column ORDER BY sorting_column`.

In [ ]:
%%sql

SELECT employee_id, department_id, salary,
    count(1) OVER (PARTITION BY department_id) AS employee_count,
    rank() OVER (ORDER BY salary DESC) AS rnk,
    lead(employee_id) OVER (PARTITION BY department_id ORDER BY salary DESC) AS lead_emp_id,
    lead(salary) OVER (PARTITION BY department_id ORDER BY salary DESC) AS lead_emp_sal
FROM employees
ORDER BY employee_id

## Aggregations using Windowing Functions

Let us see how we can perform aggregations with in a partition or group using Windowing/Analytics Functions.

* For simple aggregations where we have to get grouping key and aggregated results we can use **GROUP BY**.
* If we want to get the raw data along with aggregated results, then using **GROUP BY** is not possible or overly complicated.
* Using aggregate functions with **OVER** Clause not only simplifies the process of writing query, but also better with respect to performance.
* Let us take an example of getting employee salary percentage when compared to department salary expense.


> Let us see how we can get it using Analytics/Windowing Functions. 

* We can use all standard aggregate functions such as `count`, `sum`, `min`, `max`, `avg` etc.

## Using LEAD or LAG

Let us understand LEAD and LAG functions to get column values from following or prior records.

## Ranking using Windowing Functions

Let us see how we can assign ranks using different **rank** functions.
* If we have to assign ranks globally, we just need to specify **ORDER BY**
* If we have to assign ranks with in a key then we need to specify **PARTITION BY** and then **ORDER BY**.
* By default **ORDER BY** will sort the data in ascending order. We can change the order by passing **DESC** after order by.
* We have 3 main functions to assign ranks - `rank`, `dense_rank` and `row_number`. We will see the difference between the 3 in a moment.

Here is an example to assign sparse ranks using the table daily_product_revenue with in each day based on revenue. We can use `rank` function to assign sparse ranks.

Let us understand the difference between **rank**, **dense_rank** and **row_number**.

* We can use either of the functions to generate ranks when there are no duplicates in the column based on which ranks are assigned.
* When the column based on which ranks are assigned have duplicates then row_number should not be used as it generate unique number for each record with in the partition. For those duplicate values, the row number need not be same across multiple runs.
* **rank** will skip the ranks in between if multiple people get the same rank while **dense_rank** will not skip the ranks based up on the number of times the value is repeated.

## Order of execution of SQL

.

1. **FROM**
2. **JOIN** or **OUTER JOIN** with **ON**
3. **WHERE**
4. **GROUP BY** and optionally **HAVING**
5. **SELECT**
6. **ORDER BY**

As **SELECT** is executed before **ORDER BY** clause, we will not be able to refer the aliases defined in **SELECT** caluse in other clauses except for **ORDER BY** in most of the traditional databases. However, in Spark we can specify the aliases defined in **SELECT** in **HAVING** as well as **ORDER BY**.



## Overview of Sub Queries

Let us recap about Sub Queries.
* We typically have Sub Queries in **FROM** Clause.
* We need not provide alias to the Sub Queries in **FROM** Clause in Spark SQL. In earlier versions, you might have to provide alias for the Sub Query.
* We use Sub Queries quite often over queries using Analytics/Windowing Functions

## Filtering - Window Function Results

Let us understand how to filter on top of results of Window Functions.
* We can use **Window Functions** only in **SELECT** Clause.
* If we have to filter based on Window Function results, then we need to use Sub Queries.
* Once the query with window functions is defined as sub query, we can apply filter using aliases provided for the Window Functions.

Here is the example where we can filter data based on Window Functions.

### Ranking and Filtering - Recap

Let us recap the procedure to get top 5 orders by revenue for each day.

* We have our original data in **orders** and **order_items**
* We can pre-compute the data and store in a table or create a view with the logic to generate **daily product revenue**
* Then, we have to use the view or table or even sub query to compute rank
* We can use the query with ranks as sub query to filter so that we can get top 5 products by revenue.
* Let us see the overall process in action.

Let us come up with the query to compute daily product revenue.

## Cumulative or Moving Aggregations

Let us understand how we can take care of cumulative or moving aggregations using Spark SQL.
* When it comes to Windowing or Analytic Functions we can also specify window using `ROWS BETWEEN` clause.
* We can leverage `ROWS BETWEEN` for cumulative aggregations or moving aggregations.
* Here is an example of cumulative sum.

### question of moving sum 


In [ ]:
spark.sql("""
SELECT t.*,
    round(sum(t.revenue) OVER (
        PARTITION BY date_format(order_date, 'yyyy-MM')
        ORDER BY order_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ), 2) AS cumulative_daily_revenue
FROM daily_revenue t
ORDER BY date_format(order_date, 'yyyy-MM'), 
    order_date
""").
    show(100, false)

In [ ]:
%%sql

SELECT t.*,
    round(sum(t.revenue) OVER (
        PARTITION BY date_format(order_date, 'yyyy-MM')
        ORDER BY order_date
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ), 2) AS moving_3day_revenue
FROM daily_revenue t
ORDER BY date_format(order_date, 'yyyy-MM'),
    order_date